# DS-2.2 Baseline modeling pt.2

## **Description:**

Time to create wonders!! We are finally ready to produce our first ML model to meet customer requirements:
- Create your feature extraction step in accordance with our validation schema
- Intergrate your extraction algo into your validation class to ensure data leakage absence
- Create several ML models: 
    - Linear regression, SVR, Random Forest, XGBoost and play with them, find the best model and make your first submission

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../')
from scripts.etl import transform_df_types
import scripts.baseline_modeling as bm # baseline_modeling.py module